# Design and Analysis of Possible Data Structures for L3/L7 Routing

The goal is to adapt some popular data structures (Buffalo, SetSep, Cuckoo, and Othello) for L3/L7 routing and discover their advantages and disadvantages on the following aspects: 

- Memory footprint
    - Best case memory occupation
    - Actual memory occupation due to the uncontinuous enlarge and shrink algorithm
- Max possible throughput
    - Theoretical analysis in memory reads/writes and hash times per lookup
    - Actual throughput on commodity servers
- Dynamic performances
    - Construction time
    - Update upon routing changes 
        - Latency
        - Amount of data sent
- Routing accuracy and redundancy analysis
    - False positive ratio and false negtive ratio
    - Distribution, expectation, and standard error of: 
        - routing errors 
        - routing stretches (in the sense of unnecessary routing)
    - Simulation results of the above

**_TODO_**: How to specify the network change? Just a node failure or a link failure? 

**_Note for myself_**: The application here is routing, not load balancing. The state is routing rules; the key - value pairs are *resource ID - port number*. The traffic distribution only affects the hotspot throughput and the overall throughput, not the memory footprint. 

## Scenario assumptions

There are a total number of $n$ nodes (in L3, routers; in L7, clients) in the network. All nodes in both scenarios are controlled by a central controller to manage the routing rules. The controller is notified with the current network topology and cost matrix. 

The resources are being added and removed frequently, so the design of the query structure should also take the dynamics into consideration. 

The distribution of traffics over the nodes are examined in three types: Normal distribution ($N$, $\sigma$), Uniform distribution, and Exponential distribution ($\lambda$). 

All hash functions sampled from the hash function family are uniformly random over their range. 

The resource ID being queried may be legal, and may be alien. 

**_NOTE_**: As the distribution is over the node indices, **merely** normal distribution is not a precise description of the traffic distribution, because the hotsopts may gather or may be more evenly distributed, and because each traffic has both a source and a target. 

**_TODO_**: how to specify the precise distribution, for source and destination? Possible solution: 
Distribute traffics among the indices and uniformly randomly assign the indices to nodes. For the source and destination distribution type: 

- Maybe the same (Normal-Normal, Uniform-Uniform, Exponential-Exponential)
- Maybe all possible combinations (Normal-Normal, Normal-Uniform, ...)


1. L3 network
    - A very limited number ($n_r,\ 1 \le n_r \ll n$) of nodes are possible entrance gateways and the gateways have a relatively higher memory capacity and computation power
    - The nodes are sparcely connected, which means the adjacent matrix is a sparse matrix
    - Traffics are from outside of the network and both incoming and outgoing traffics must pass the gateway in the routing

1. L7 network
    - All nodes are fully connected to form a complete graph
    - Each node is both the possible consumer and the possible generator of traffics / messages
    - The keys are resource ID. The number of resources are far more than the number of servers
    
**_TODO_**: how to generate network? What is the assumption of the topology of the L3/L7 network? 

1. Other parameters
    - Key: resource ID, maybe server IP address, server index, resource ID. Number: $n_k$. Length: $l_k$, different in L3 and L7 networks.
    - Value: port number. Number: $n_p$. Length: $l_p = \lceil log_2(n_p) \rceil$. 
    - Cache line size: $l_c$. Typical number is $64 \times 8 = 512\ bits$

**_Note_**:

1. Lookup structures are all the same for all nodes in the network. But the values are different for a same key. 
1. The equation $l_k = \lceil log_2(n_k) \rceil$ may not hold. 

----------------------------------



----------------------------------

# Cuckoo based solutions

System parameters:

- Load factor: $r_l=0.83,\ e_l=1/r_l = 1.20$
- Hashes (buckets) for a key: $n_b=2$
- Slots in a bucket: $n_s=4$

--------------

## Naive solution: just store all possible key-value pairs

A slot: Resource ID || Port number. Slot length:
$$l_s=l_k+l_p$$
Bucket length:
$$l_b=l_s \cdot n_s$$
The emptiness is expressed by a special port number. 

### Memory footprint

Minimal memory occupation on a single node: $$s=l_s \cdot n_k \cdot e_l$$

#### Best case memory occupation

$$M_{best} = s \cdot n $$

#### Actual memory occupation in practice
The expansion factor due to the alignment of the bucket to the cache line:
$$e_b = \frac{\lceil n_s \cdot l_s / l_c \rceil \cdot l_c}{n_s \cdot l_s}  $$
The expansion factor due to uncontinous data structure expansion algorithm: 
$$e_a = \frac {2^{\lceil log_2 \lceil n_k \cdot e_l / n_s \rceil \rceil}}{n_k \cdot e_l / n_s}$$

**_Note_**: if the query structure is updated very rarely, then binary expansion is not needed. 
So the actual memory occupation:
$$M_{actual} = e_a \cdot e_b \cdot s \cdot n = e_a \cdot e_b \cdot M_{best} $$

**_Note_**: if align slot to byte, only the computation power is saved, and the number of memory accesses may well increase. Simply pad $l_s$ can adapt the following equations to that scenario. 

### Max possible throughput
#### Theoretical analysis in the number of memory bus accesses and the number of hashed bits per lookup

- If align bucket to cache line:
    The expected number of memory bus accesses:
    $$E(C_{read}) = \lceil l_k/l_c \rceil + \sum_{i=1}^{n_b \cdot n_s} \frac{ \lfloor i/n_s \rfloor \cdot \lceil n_s \cdot l_s / l_c \rceil + \lceil (i \mod n_s) \cdot l_s/l_c \rceil }{n_b \cdot n_s}$$
    The expected number of hashed bits:
    $$E(L_{hash}) = 1.5 \cdot l_k$$

- If do not align bucket (always align the queried key to cache line):
Assuming the bucket is aligned to the cache line at the first element. Then the Greatest Common Divider (gcd) of the length of the cache line and the length of the bucket is the step of possible relative positions. (Need more description)
$$l_g=gcd(l_c,l_b)$$
The extra memory read happens when the remainder of the read bits is distributed in two cache lines (need a picture here). The remainder of $l$ bits:
$$l_r(l) = \lceil (l \mod l_c) / l_g \rceil \cdot l_g$$
The expected memory reads for the first $i$ slots in a bucket: 
$$E(C_s^i) = \lceil (i \cdot l_s)/l_c \rceil + 
\begin{cases}
    \frac{l_r(i\cdot l_s) - l_g}{l_c},& l_r(i\cdot l_s) \ne 0 \\
    0, & \text{otherwise}
\end{cases}$$
The expected memory reads for a whole bucket: 
$$E(C_b) = E(C_s^{n_s})$$
The expected number of memory bus accesses:
$$E(C_{read}) = \lceil l_k/l_c \rceil + \sum_{i=1}^{n_b \cdot n_s} \frac{ \lfloor i/n_s \rfloor \cdot E(C_b) + E(C_s^i) }{n_b \cdot n_s}$$
The expected number of hashed bits:
$$E(L_{hash}) = 1.5 \cdot l_k $$

**_Note_**: It is very hard to model cache behaviors into the above memory access equations. Instead, we are assuming no cache-hit happens. This is reasonable because 1) the hash is random and 2) the number of IDs are large so that the cache is always loading new buckets. 

#### Actual throughput on commodity servers
To be done

### Dynamic performances
#### Construction time

#### Update upon routing changes
- Latency
- Amount of data sent

------------------------------------



------------------------------------

## Smarter Cuckoo: exact match for L3, best effort filter for L7

### For L3 gateway nodes:
A slot: Resource ID || Port number. 
$$l_s=l_k+l_p$$

### For all other nodes (L3 internal nodes and all L7 nodes):
- Use the multi-level-table design to avoid key digest collision
- A slot: Resource digest || Port number. Define resource digest length: $l_d$
$$l_s=l_d+l_p$$

Bucket length:
$$l_b=l_s \cdot n_s$$

**_Note_**: different $l_d$ at different levels may be chosen. Define $l_{d_k}$

### Memory footprint
#### Actual memory occupation in practice for non-gateway nodes

Let: 
<!---- $p_k$: the probability of "two uniformly randomly chosen resource IDs $k_1$ and $k_2$ will collide at level-k"  -->

- $C_k$: the expected number of (collision-free) elements at level-k
- $N_{b_k}$: the expected total number of buckets
- $N_k$: the expected number of input elements to level-k (the number of elements that collide with one or more other elements at all previous levels)
- $R_k$: the expected actual load ratio at level-k
The above variables approximately subject to the following equation set:
$$
\begin{array}{rcl}
N_{b_k} =& 2^{\lceil log_2(\lceil C_k \cdot e_l / n_s \rceil) \rceil}\\
R_k     =& \frac {C_k} {N_{b_k} \cdot n_s} \\
%p_k     =& \frac {N_{b_k} + 2 \cdot 2 \cdot N_{b_k} (N_{b_k} - 1) + 2 \cdot N_{b_k} (N_{b_k} - 1) + 4 \cdot N_{b_k} (N_{b_k} - 1) (N_{b_k} - 2) }{2^{l_{d_k}} \cdot N_{b_k}^4 } \\
%       =& \frac {1 + 6 \cdot (N_{b_k} - 1) + 4 \cdot (N_{b_k} - 1) (N_{b_k} - 2) }{2^{l_{d_k}} \cdot N_{b_k}^3 } \\
N_k     =& n_k - \sum_{i=1}^{k-1} C_k \\
C_k     =& (\sum_{i=1}^{n_b} \frac{P_{N_b}^i}{{N_b}^{n_b}} (1-\frac{1}{2^{l_{d_k}}})^{i \cdot n_s \cdot R_k} )N_k 
\end{array}
$$
where $k \in \{1,2\}$

**_Note_**: 

1. We assume self digest collision on the two hash functions can happen: $h_1(k_i) = h_2(k_i)$, for $i=1,2$, because for the higher level tables, there may be very few buckets, even one bucket. 
1. Despite of the special treatment to the self digest collisions, this equation set is still approximate. Because we are assuming here the expectation can propagate beyound non-linear operators, the error bound of which actually requires a proof. 
1. Even it is an approximation, this equation set is very hard to solve. However, the result for level 1 is accurate because $N_1$ is $n_k$, not an approximation. 



-----------------------------------

## Even smarter Cuckoo: Cuckoo Filter Adaptation
As the quantitative analysis is very hard to perform for the above design while the original idea is to filter most keys at the first level and cope with few collisions at the second level, the design can be changed to let the level-2 store the full key, not the digest. A very precise expactation can be derived for this design. 

This approach is actually an adaptation of Cuckoo Filter. However, due to the discrepancy in the goals of a filter and a switch, there are some very important differences between this adaptation and the original design of Cuckoo Filter: 

- As the Cuckoo Filter is only used as a query data structure and there is a control plane responsible for maintaining the whole key-value mapping and for constructing the query data structures, there is no need to use partial-key cuckoo hashing to determine the two buckets, just two normal hash functions suffice. And this way is better because the digest length becomes totally independent from the key set size. 
- The original Cuckoo Filter is used for *approximate set membership* test while in this scenario, the adaptation should make decision on forwarding port number, the key digest duplication is not permitted in a bucket or it will be impossible to decide the correct port number associations for the two keys. So the level-2 works as a buffer to handle all collided keys. 

### For L3 gateway nodes:
A slot: Resource ID || Port number. 
$$l_s=l_k+l_p$$

### For all other nodes (L3 internal nodes and all L7 nodes):

Use the 2-level-table design to avoid key digest collision. 

- For the first level, a slot: Resource digest || Port number. Define resource digest length: $l_d$. 
$$l_{s1}=l_d+l_p$$
- For the second level, a slot: Resource ID || Port number. 
$$l_{s2}=l_k+l_p$$

### Memory footprint
#### Actual memory occupation in practice for non-gateway nodes


Let: 

- $N_1$: the expected number of (collision-free) elements at level-1
- $N_b$: the expected total number of buckets at level-1
- $R_l$: the expected actual load ratio at level-1
- $C_b$: the expected possibility of "there will be no collision if another key is put into one bucket"

The above variables approximately satisfy the following equation set:
$$
\begin{array}{rcl}
N_b  =& 2^{\lceil log_2(\lceil N_1 \cdot e_l / n_s \rceil) \rceil}\\
R_l  =& \frac {N_1} {N_b \cdot n_s} \\
N_1  =& (\sum_{i=1}^{n_b} \frac{P {N_b \choose i}}{{N_b}^{n_b}} C_b^i ) n_k \\
C_b  =& \sum_{i=0}^{n_s} (1-\frac{1}{2^{l_d}})^i \cdot (1-R_l)^{n_s-i} R_l^i {n_s \choose i}
\end{array}
$$
So, the expected elements in level-2: 
$$N_2 = n_k -N_1$$
The expansion factor due to the alignment of the bucket to the cache line for level-1:
$$e_{b1} = \frac{\lceil n_s \cdot l_{s1} / l_c \rceil \cdot l_c}{n_s \cdot l_{s1}}  $$
The expansion factor due to the alignment of the bucket to the cache line for level-2:
$$e_{b2} = \frac{\lceil n_s \cdot l_{s2} / l_c \rceil \cdot l_c}{n_s \cdot l_{s2}}  $$
The expansion factor due to uncontinous data structure expansion algorithm for level-2: 
$$e_a = \frac {2^{\lceil log_2 \lceil N_2 \cdot e_l / n_s \rceil \rceil}}{N_2 \cdot e_l / n_s}$$


And the total memory consumption is:
$$
M_{actual} = M_1 + M_2 = (N_b \cdot n_s \cdot l_{s1} \cdot e_{b1}) + (e_a \cdot N_2 \cdot l_{s2} \cdot e_{b2})
$$

### Max possible throughput
#### Theoretical analysis in the number of memory bus accesses and the number of hashed bits per lookup

- If align bucket to cache line: <br>
    Define the function: 
    $$f(i, l_s) =  \lfloor i/n_s \rfloor \cdot \lceil n_s \cdot l_s / l_c \rceil + \lceil (i \mod n_s) \cdot l_s/l_c \rceil$$
    The value of $f(i, l_s)$ is the expected memory bus read operations for a query if $i$ slots are examined, where the slot length is $l_s$. 
    
    The expected number of memory bus accesses:
    $$E(C_{read}) = \lceil l_k/l_c \rceil + \frac{N_1} {n_k} \sum_{i=1}^{n_b \cdot n_s} \frac{f(i, l_{s1}) }{n_b \cdot n_s} + \frac{N_2} {n_k} (f(n_b\cdot n_s, l_{s1})  +\sum_{i=1}^{n_b \cdot n_s} \frac{f(i, l_{s2}) }{n_b \cdot n_s} )$$
   
- If do not align bucket (always align the queried key to cache line):<br>
    The calculation is simillar to the above. Refer "Naive Cuckoo".
    
The expected number of hashed bits:
    $$E(L_{hash}) = l_k + \frac{N_1}{n_k} (1.5 \cdot l_k) + \frac{N_2}{n_k} (2 \cdot l_k + 1.5 \cdot l_k) = (\frac{2N_2}{n_k} + 2.5) \cdot l_k$$

#### Actual throughput on commodity servers
To be done

### Dynamic performances
#### Construction time

#### Update upon routing changes
- Latency
- Amount of data sent

#### Elien key in L7 networks
As there are some "full equipped" gateways in an L3 network, no alien resource requests may be routed into the network. But things are different in an L7 network: some client application may encounter a function failure or may be malicious and sends out an alien resource request. Upon receiving the alien request, it is best to detect it at the first hop, or there will be a wrong path of length one. 

- Number of memory bus accesses
$$E(C_{read}) = \lceil l_k/l_c \rceil + f(n_b\cdot n_s, l_{s1}) + f(n_s \cdot n_b, l_{s2}) $$
- Hash length
$$E(L_{hash}) = l_k + (2 \cdot l_k + 2 \cdot l_k) = 5 l_k$$
- False Positive possibility
$$FP = 1 - C_b  $$

----------------------------------



----------------------------------

# Othello based solutions

System parameters:

- Expansion factor: $$e_a = 1.0,\ e_b = 1.33$$
--------------

## Othello (plain)
Like Cuckoo Filter, for L3 networks, the gateways contains full key-value pairs in the form of *OthelloMap*, and the internal nodes contains Othello. For L7 networks, only Othello is used, and alien packets are forwarded at most one hop. 

### For L3 gateway nodes:
A slot: index into the key-value array. Define $l_s$ is the length of the index. Ideally,  
$$
l_s = \lceil log_2\, n_k \rceil
$$

### For all other nodes (L3 internal nodes and all L7 nodes):

Unlike Cuckoo, Othello does not have false positives. So the plain Othello is sufficient for valid keys. A slot is only the port number. 
$$
l_s = l_p
$$

### Memory footprint
#### Actual memory occupation in practice for non-gateway nodes

Let: 

- The expansion factor due to the alignment of the slot to the cache line:
$$
e_c = \frac{l_c}{\lfloor l_c / l_s \rfloor \cdot l_s} 
$$
- The real expansion factor of array $A$ and $B$ due to uncontinous data structure expansion algorithm: 
$$E_a = \frac {2^{\lceil log_2 \, n_k \cdot e_a  \rceil}}{n_k \cdot e_a}$$
$$E_b = \frac {2^{\lceil log_2 \, n_k \cdot e_b  \rceil}}{n_k \cdot e_b}$$
- The number of slots of array A and array B in Othello query structure:
$$
m_a = E_a \cdot n_k 
$$
$$
m_b = E_b \cdot n_k
$$
**_Note_**: the expansion factor due to the alignment of slot to byte is easily modeled by padding the slots. 

And the overall memory consumption is:
$$
M_{actual} = e_c \cdot ( m_a + m_b ) \cdot l_s
$$

### Max possible throughput
#### Theoretical analysis in the number of memory bus accesses and the number of hashed bits per lookup

- If align slots to cache lines: <br>
$$
E(C_{read}) = \lceil l_k / l_c \rceil + 2
$$

- If do not align slots (always align the queried key to cache line):<br>
$$
l_g = gcd(l_s, l_c)
$$
$$
E(C_{read}) = \lceil l_k / l_c \rceil + 2 \cdot (1 + \frac{l_s-l_g}{l_c})
$$
    
The expected number of hashed bits:
    $$E(L_{hash}) = 2l_k$$

#### Actual throughput on commodity servers
To be done

### Dynamic performances
#### Construction time

#### Update upon routing changes
- Latency
- Amount of data sent

#### Elien key in L7 networks
- Number of memory bus accesses: The same with the above result
- Hash length: The same with the above result
- False Positive possibility
$$FP = 1  $$

----------------------------------


----------------------------------

# Change scenario
On one hand, introducing the gateway really simplifies the design and analysis. How about removing this assumption? And the false positive ratio is very high for L7 network of the plain Othello design. 

## New scenario assumption
- Copy - paste the old assumptions. 
- All nodes behave the same. That is, no gateway is allowed, the alien keys should be treated at every node. 
- As the memory is limited while the key size is very big, and as we really want to make the query for valid keys faster, designs with less memory spent on storing keys are preferred. 

----------------------------------

## Cuckoo Design
As storing keys maybe very expensive and a small length of key digests for Cuckoo Filter is sufficient for arbitrarily large of key size (according to the figures), the design of Cuckoo based solution is exactly the same as the Cuckoo Filter. There will be no false routing for valid keys. But as for the alien keys, this approach may wrongly math them with existing key digests and thus introduce an "error path", or even "error loop". 

The false positive only happens when there is a match of the digests at the first level. Because the second level stores the whole keys, no false positive is possible. 

### Alien keys
The FP ratio at one hop: 
$$FP = 1 - C_b  $$
Where $C_b$ is defined in Cuckoo Filter. 

Generally speaking, the possibility of the wrong path to form a loop depends on the nework topology. But for Cuckoo Filter particularly, there is no possibility forming a loop. Because the routing table construction at the control plane is optimized so that the same key digest function is used for all nodes and thus if there is one false positive for the alien key, then the alien key will follow the same path with the valid key that collides with it. So the expected error path length: 
$$ L_e = FP \cdot E(L) $$
where $E(L)$ is the expected length of a random path. 

-------------------------------------

## Othello Design
As the plain Othello has a 100% false positive ratio, there should be some checksum to filter out alien keys. The checksum comes in two ways: 

- An additional bit in every slot denoting the emptiness of the slot. If there is no edges linking current slot, this bit is set to 0. This is the original idea. When taken into practice, this one-bit cost can be saved while keeping the ability of telling the empty slots from the non-empty ones. Just let the port number 0 to denote the virtual port to drop packets, and let the all zero slots to denote a white slot. This whiteness marking is easily achieve by assuring the first "free end" of the connected component is not all zero, then the all zero slots will never show up in the middle, because all values for valid keys are not zero. 
- Adding a checksum array $C$, $m_c=m_a$. Then the routing lookup workflow becomes: 
    - Hash the key three times for slot indices $i_a$, $i_b$, and $i_c$. 
    - Hash the key another time to get the digest $d_k$
    - Get the slots $s_a=u_a$, $s_b=u_b$, $s_c = d$
    - If $u_b \oplus d = d_k$, then the value is $u_b \oplus u_a$ (false positive may be introduced here), or the key must be an alien key. 

A slot in array $A$: port number value. $$l_{sa} = l_p$$
A slot in array $B$: generic value. $$l_{sb} = max(l_p, l_d)$$
A slot in array $C$: digest value. $$l_{sc} = l_d$$

### Memory footprint
#### Actual memory occupation in practice for a node

Let: 

- The expansion factor due to the alignment of the slot to the cache line:
$$
e_{ca} = \frac{l_c}{\lfloor l_c / l_{sa} \rfloor \cdot l_{sa}} 
$$
$$
e_{cb} = \frac{l_c}{\lfloor l_c / l_{sb} \rfloor \cdot l_{sb}} 
$$
$$
e_{cc} = \frac{l_c}{\lfloor l_c / l_{sc} \rfloor \cdot l_{sc}} 
$$
- The number of slots of array A and array B in Othello query structure:
$$
m_c = m_a = e_a \cdot n_k 
$$
$$
m_b = e_b \cdot n_k
$$

And the overall memory consumption is:
$$
M_{actual} = e_{ca} \cdot m_a \cdot l_{sa} + e_{cb} \cdot m_b \cdot l_{sb} + e_{cc} \cdot m_c \cdot l_{sc}
$$

### Max possible throughput
#### Theoretical analysis in the number of memory bus accesses and the number of hashed bits per lookup

- If align slots to cache lines: <br>
$$
E(C_{read}) = \lceil l_k / l_c \rceil + 2
$$
- If do not align slots (always align the queried key to cache line):<br>
$$
l_{ga} = gcd(l_{sa}, l_c)
$$
$$
l_{gb} = gcd(l_{sb}, l_c)
$$
$$
l_{gc} = gcd(l_{sc}, l_c)
$$
$$
E(C_{read}) = \lceil l_k / l_c \rceil + 3 + \frac{l_{sa}-l_{ga}}{l_c} + \frac{l_{sb}-l_{gb}}{l_c} + \frac{l_{sc}-l_{gc}}{l_c}
$$
    
The expected number of hashed bits:
    $$E(L_{hash}) = 4l_k$$

#### Actual throughput on commodity servers
To be done

### Dynamic performances
#### Construction time

#### Update upon routing changes
- Latency
- Amount of data sent

#### Elien keys
- Number of memory bus accesses: The same with the above result
- Hash length: The same with the above result
- The expected ratio of empty slots in $A$: 
$$\epsilon_a = (\frac{m_a-1}{m_a})^{n_k} \approx e^{-\frac{n_k}{m_a}} $$
- The expected ratio of empty slots in $B$: 
$$\epsilon_b =  (\frac{m_b-1}{m_b})^{n_k} \approx e^{-\frac{n_k}{m_b}} $$
- False Positive possibility
$$FP = \frac{1}{2^{l_d}-1} \cdot (1 - \epsilon_a) \cdot (1 - \epsilon_b)  $$
- Loop possibility
Unlike Cuckoo, there is no level-2 with full key information. Like Cuckoo, the false positive will always be the false positive and thus no loop is possible for both valid keys and alien keys. So the expected error path length: 
$$ L_e = FP \cdot E(L) $$
where $E(L)$ is the expected length of a random path. 


# Design for filters
## Scenario assumptions
- There is a control plane managing full states. No dynamics in the query data structure. 
- 


## Othello Filter
Plain Othello with digest. A slot: digest. Number of slots: $m_a = n_k$, $m_b = 1.33n_k$.
As the filter are updated by the control plane, no padding to power of 2 is needed. 
- The expected ratio of empty slots in $A$: 
$$\epsilon_a = (\frac{m_a-1}{m_a})^{n_k} \approx e^{-\frac{n_k}{m_a}} $$
- The expected ratio of empty slots in $B$: 
$$\epsilon_b =  (\frac{m_b-1}{m_b})^{n_k} \approx e^{-\frac{n_k}{m_b}} $$
- False Positive possibility
$$FP = \frac{1}{2^{l_d}-1} \cdot (1 - \epsilon_a) \cdot (1 - \epsilon_b)  $$

## Cuckoo Filter
- No need for deletion and insertion support, so $l_d$ is independent from other parameters. 
- A slot: Resource digest. Define resource digest length: $l_d$
$$l_s=l_d$$

Bucket length:
$$l_b=l_s \cdot n_s$$

### Memory footprint
#### Actual memory occupation in practice for non-gateway nodes
Expansion 
$$  $$

In [10]:
%matplotlib inline
import math
import re
from sympy import list2numpy 
import matplotlib.pyplot as plt

lc = 512

lkMaxMax = 1024
lkMaxMin = 32

lkMax = 1024
lpMax = 12
nkMax = 2 ** 22
nMax = 4096

lpMin = 1
nkMin = 1
nMin = 1

l3 = True
alien = False
alignToCacheLine = False
alignSlotToByte = False
allowGateway = False

compact = True

nk = 1024 * 1024
lk = 1020
lp = 5

lkMin = math.ceil(math.log2(nk))

n = 1024

nbCuckoo = 2
nsCuckoo = 4
elCuckoo = 1.05

ldCuckoo = 8
ldCuckooMax = 16
ldCuckooMin = 1

ldOthello = 8
ldOthelloMax = 16
ldOthelloMin = 0

eaOthello = 1.33
ebOthello = 1.0

targetFP = 20
targetFPMax = 100
targetFPMin = 1


def mergeWithGlobal(**kwargs):
  ''':return "n", "logNk", "lk", "lp", "alignBucketToRead", "alignSlotToByte", "l3"'''
  return [kwargs[var] if var in kwargs.keys() else globals()[var]
          for var in ["n", "logNk", "lk", "lp", "alignBucketToRead", "alignSlotToByte", "l3"]]


def combinationNumber(n, k):
  result = 1
  for i in range(k):
    result *= (n - i) / (i + 1)
  return result


def permutationNumber(n, k):
  result = 1
  for i in range(k):
    result *= (n - i)
  return result


def camel_case_split(identifier):
  return re.sub('(?!^)([A-Z][^A-Z]+)', r' \1', identifier).split()


def type_of_script():
  try:
    ipy_str = str(type(get_ipython()))
    if 'zmqshell' in ipy_str:
      return 'jupyter'
    if 'terminal' in ipy_str:
      return 'ipython'
  except:
    return 'terminal'


def in_jupyter():
  return type_of_script() == 'jupyter'


class Base:
  def getExtraRead(self, x="lk"):
    global alignToCacheLine
    oldAlign = alignToCacheLine

    alignToCacheLine = False
    unaligned = self.getRead(x)
    alignToCacheLine = True
    aligned = self.getRead(x)

    alignToCacheLine = oldAlign
    return list2numpy(unaligned) - list2numpy(aligned)

  def getMemoryFootPrint(self, x="lk"):
    pass

  def getRead(self, x="lk"):
    pass

  def getHashTimes(self, x="lk"):
    pass

  def getFalsePositive(self, x="lk"):
    pass

  def calcMemoryFootprint(self, **params):
    pass

  def calcMemoryLoads(self, **params):
    pass

  def calcHashTimes(self, **params):
    pass


class BloomForwarder(Base):
  def calc(self, nk, lp, fp):
    np = math.ceil(2 ** lp)
    fpSingle = 1 - (1 - fp) ** (1 / np)

    nk = nk / np

    K = 8
    m = 8 * nk
    FPSingle = 0.1

    oldM = 0
    oldK = 0
    oldFPSingle = 0.1

    while oldK != K or oldM != m or (FPSingle - fpSingle) / fpSingle > 0.01:
      oldK = K
      oldM = m
      oldFPSingle = FPSingle

      K = max(int(math.log(2) * m / nk + 0.5), 1)
      m = int(-K * nk / math.log(1 - fpSingle ** (1 / K)))

      p = (1 - 1 / m) ** (K * nk)
      FPSingle = (1 - p) ** K

      if (oldK == K + 1 or oldK == K - 1) and math.fabs((FPSingle - fpSingle) / fpSingle) <= 0.01 and m < oldM:
        break

    FP = 1 - (1 - FPSingle) ** np

    Nt = K * p ** (K - 1)
    for i in range(1, K):
      Nt += i * (1 - p) * p ** (i - 1)

    return m * np, K, p, Nt, FP

  def calcMemoryFootprint(self, **params):
    nk = params['nk']
    lp = params['lp']
    fp = params['fp']

    return self.calc(nk, lp, fp)[0]

  def calcFalsePositive(self, **params):
    nk = params['nk']
    lp = params['lp']
    m = params['m']

    np = math.ceil(2 ** lp)

    m /= np
    nk /= np

    K = math.log(2) * m / nk

    FPSingle = 2
    for k in (math.ceil(K), math.floor(K)):
      FPSingle = min(FPSingle, (math.e ** (-nk * k / m)) ** k)

    FP = 1 - (1 - FPSingle) ** np

    return FP

  def getMemoryFootPrint(self, x="lk"):
    fp = 0.0001 * math.e ** (targetFP / 99 * math.log(100))

    def calc(nk, lk, lp):
      m, K, p, Nt, FP = self.calc(nk, lp, fp)
      return m

    return [calc(nk if x is not "nk" else i, lk if x is not "lk" else i, lp if x is not "lp" else i)
            for i in range(globals()[x + "Min"], globals()[x + "Max"] + 1)]

  def calcMemoryLoads(self, **params):
    return math.ceil(lk / lc) + self.calcHashTimes(**params)

  def calcHashTimes(self, **params):
    nk = params['nk']
    lp = params['lp']
    fp = params['fp']
    np = math.ceil(2 ** lp)

    m, K, p, Nt, FP = self.calc(nk, lp, fp)

    if l3 and allowGateway or not alien:
      return K + Nt / 2 * (np - 1)
    else:
      return np * Nt

  def getRead(self, x="lk"):
    fp = 0.0001 * math.e ** (targetFP / 99 * math.log(100))

    def calc(nk, lk, lp):
      np = math.ceil(2 ** lp)
      m, K, p, Nt, FP = self.calc(nk, lp, fp)

      return math.ceil(lk / lc) + K + Nt / 2 * (np - 1)

    return [calc(nk if x is not "nk" else i, lk if x is not "lk" else i, lp if x is not "lp" else i)
            for i in range(globals()[x + "Min"], globals()[x + "Max"] + 1)]

  def getHashTimes(self, x="lk"):
    fp = 0.0001 * math.e ** (targetFP / 99 * math.log(100))

    def calc(nk, lk, lp):
      m, K, p, Nt, FP = self.calc(nk, lp, fp)
      np = math.ceil(2 ** lp)

      if l3 and allowGateway or not alien:
        return K + Nt / 2 * (np - 1)
      else:
        return Nt / 2 * (np + 1)

    return [calc(nk if x is not "nk" else i, lk if x is not "lk" else i, lp if x is not "lp" else i)
            for i in range(globals()[x + "Min"], globals()[x + "Max"] + 1)]

  def getFalsePositive(self, x="lk"):
    fp = 0.0001 * math.e ** (targetFP / 99 * math.log(100))

    def calc(nk, lk, lp):
      m, K, p, Nt, FP = self.calc(nk, lp, fp)

      return FP

    return [calc(nk if x is not "nk" else i, lk if x is not "lk" else i, lp if x is not "lp" else i)
            for i in range(globals()[x + "Min"], globals()[x + "Max"] + 1)]


class CuckooForwarder(Base):
  @staticmethod
  def Eeta(digestLength):
    if digestLength > 16: return 1
    return \
      [0, 0.0005001000000000033, 0.022338599999999997, 0.14957240000000002, 0.38679990000000003, 0.6217775999999999,
       0.7885496000000001, 0.8880863999999999, 0.9423754000000001, 0.9707575600000002, 0.9852790700000001,
       0.99260854, 0.99628573, 0.9981500799999999, 0.99907092, 0.99953898, 0.9997694600000001][digestLength - 1]

  def calc(self, nk, lk, lp, ld):
    ls1 = ld + lp
    ls2 = lk + lp
    lb1 = ls1 * nsCuckoo
    lb2 = ls2 * nsCuckoo

    if alignSlotToByte:
      ls1 = (ls1 + 7) // 8 * 8
      ls2 = (ls2 + 7) // 8 * 8

    N1 = int(CuckooForwarder.Eeta(int(max(min(16, ld), 1))) * nk)
    N2 = nk - N1

    eb1 = math.ceil(nsCuckoo * ls1 / lc) * lc / (nsCuckoo * ls1) if alignToCacheLine else 1
    ea1 = 1 if compact else 2 ** math.ceil(math.log2(math.ceil(N1 * elCuckoo / nsCuckoo))) * nsCuckoo / N1 / elCuckoo
    if N2 > 0:
      eb2 = 1
      ea2 = 1 if compact else 2 ** math.ceil(math.log2(math.ceil(N2 * elCuckoo / nsCuckoo))) * nsCuckoo / N2 / elCuckoo
    else:
      eb2 = 1
      ea2 = 1

    return N1, N2, ea1, eb1, ea2, eb2, ls1, ls2, lb1, lb2

  def calcMemoryFootprint(self, **params):
    nk = params['nk']
    lk = params['lk']
    lp = params['lp']
    desiredFP = params['fp']

    minMemory = 2 ** 100

    for ld in range(2, 100):
      N1, N2, ea1, eb1, ea2, eb2, ls1, ls2, lb1, lb2 = self.calc(nk, lk, lp, ld)
      memory = math.ceil(N1 * elCuckoo / nsCuckoo) * ea1 * nsCuckoo * ls1 * eb1 \
               + math.ceil(N2 * elCuckoo / nsCuckoo) * ea2 * nsCuckoo * ls2 * eb2

      fp = 1 - (1 - 2 ** (-ld)) ** (1 / elCuckoo * CuckooForwarder.Eeta(ld) * nsCuckoo * nbCuckoo)

      if fp <= desiredFP or l3 and allowGateway:
        minMemory = min(memory, minMemory)

    return minMemory

  def calcOptimalLd(self, **params):
    nk = params['nk']
    lk = params['lk']
    lp = params['lp']
    desiredFP = params['fp']

    minMemory = 2 ** 100
    minLd = -1
    for ld in range(2, 20):
      N1, N2, ea1, eb1, ea2, eb2, ls1, ls2, lb1, lb2 = self.calc(nk, lk, lp, ld)
      memory = math.ceil(N1 * elCuckoo / nsCuckoo) * ea1 * nsCuckoo * ls1 * eb1 \
               + math.ceil(N2 * elCuckoo / nsCuckoo) * ea2 * nsCuckoo * ls2 * eb2

      fp = 1 - (1 - 2 ** (-ld)) ** (1 / elCuckoo * CuckooForwarder.Eeta(ld) * nsCuckoo * nbCuckoo)

      if fp <= desiredFP or l3 and allowGateway:
        if memory < minMemory:
          minMemory = memory
          minLd = ld

    return minLd

  def getMemoryFootPrint(self, x="lk"):
    def calc(nk, lk, lp, ld):
      N1, N2, ea1, eb1, ea2, eb2, ls1, ls2, lb1, lb2 = self.calc(nk, lk, lp, ld)
      return math.ceil(N1 * elCuckoo / nsCuckoo) * ea1 * nsCuckoo * ls1 * eb1 \
             + math.ceil(N2 * elCuckoo / nsCuckoo) * ea2 * nsCuckoo * ls2 * eb2

    return [calc(nk if x is not "nk" else i, lk if x is not "lk" else i, lp if x is not "lp" else i, ldCuckoo)
            for i in range(globals()[x + "Min"], globals()[x + "Max"] + 1)]

  def calcMemoryLoads(self, **params):
    nk = params['nk']
    lk = params['lk']
    lp = params['lp']
    desiredFP = params['fp']

    for ld in range(2, 100):
      fp = 1 - (1 - 2 ** (-ld)) ** (1 / elCuckoo * CuckooForwarder.Eeta(ld) * nsCuckoo * nbCuckoo)

      if fp <= desiredFP:
        N1, N2, ea1, eb1, ea2, eb2, ls1, ls2, lb1, lb2 = self.calc(nk, lk, lp, ld)
        break

    def ECi(i, ls, lb):
      if alignToCacheLine:
        return math.ceil(i * ls / lc)
      else:
        lg = math.gcd(lc, lb)
        ll = math.ceil(i * ls % lc / lg) * lg

        return math.ceil(i * ls / lc) + ((ll - lg) / lc if ll != 0 else 0)

    if l3 and allowGateway or not alien:
      result = math.ceil(lk / lc)

      result1 = 1 / (nbCuckoo * nsCuckoo) * sum([
        (i // nsCuckoo * ECi(nsCuckoo, ls1, lb1) + ECi(i % nsCuckoo, ls1, lb1))
        for i in range(1, nbCuckoo * nsCuckoo + 1)])

      result2 = nbCuckoo * ECi(nsCuckoo, ls1, lb1) + \
                1 / (nbCuckoo * nsCuckoo) * sum([
        (i // nsCuckoo * ECi(nsCuckoo, ls2, lb2) + ECi(i % nsCuckoo, ls2, lb2))
        for i in range(1, nbCuckoo * nsCuckoo + 1)
      ])

      result += N1 / nk * result1 + N2 / nk * result2
      return result
    else:
      return math.ceil(lk / lc) + nbCuckoo * ECi(nsCuckoo, ls1, lb1) + nbCuckoo * ECi(nsCuckoo, ls2, lb2)

  def getRead(self, x="lk"):
    def calc(nk, lk, lp, ld):
      N1, N2, ea1, eb1, ea2, eb2, ls1, ls2, lb1, lb2 = self.calc(nk, lk, lp, ld)

      def ECi(i, ls, lb):
        if alignToCacheLine:
          return math.ceil(i * ls / lc)
        else:
          lg = math.gcd(lc, lb)
          ll = math.ceil(i * ls % lc / lg) * lg

          return math.ceil(i * ls / lc) + ((ll - lg) / lc if ll != 0 else 0)

      if l3 and allowGateway or not alien:
        result = math.ceil(lk / lc)

        result1 = 1 / (nbCuckoo * nsCuckoo) * sum([
          (i // nsCuckoo * ECi(nsCuckoo, ls1, lb1) + ECi(i % nsCuckoo, ls1, lb1))
          for i in range(1, nbCuckoo * nsCuckoo + 1)])

        result2 = nbCuckoo * ECi(nsCuckoo, ls1, lb1) + \
                  1 / (nbCuckoo * nsCuckoo) * sum([
          (i // nsCuckoo * ECi(nsCuckoo, ls2, lb2) + ECi(i % nsCuckoo, ls2, lb2))
          for i in range(1, nbCuckoo * nsCuckoo + 1)
        ])

        result += N1 / nk * result1 + N2 / nk * result2
        return result
      else:
        return math.ceil(lk / lc) + nbCuckoo * ECi(nsCuckoo, ls1, lb1) + nbCuckoo * ECi(nsCuckoo, ls2, lb2)

    return [calc(nk if x is not "nk" else i, lk if x is not "lk" else i, lp if x is not "lp" else i, ldCuckoo)
            for i in range(globals()[x + "Min"], globals()[x + "Max"] + 1)]

  def getHashTimes(self, x="lk"):
    def calc(nk, lk, lp, ld):
      N1, N2, ea1, eb1, ea2, eb2, ls1, ls2, lb1, lb2 = self.calc(nk, lk, lp, ld)

      if l3 and allowGateway or not alien:
        return 2 + (nbCuckoo - 1) / 2 + (1 + nbCuckoo) * N2 / nk
      else:
        return 5

    return [calc(nk if x is not "nk" else i, lk if x is not "lk" else i, lp if x is not "lp" else i, ldCuckoo)
            for i in range(globals()[x + "Min"], globals()[x + "Max"] + 1)]

  def getFalsePositive(self, x="lk"):
    def calc(nk, lk, lp, ld):
      if l3 and allowGateway:
        return 0
      else:
        return 1 - (1 - 2 ** (-ld)) ** (1 / elCuckoo * CuckooForwarder.Eeta(ld) * nsCuckoo * nbCuckoo)

    return [calc(nk if x is not "nk" else i, lk if x is not "lk" else i, lp if x is not "lp" else i, ldCuckoo)
            for i in range(globals()[x + "Min"], globals()[x + "Max"] + 1)]

  def calcHashTimes(self, **params):
    nk = params['nk']
    lk = params['lk']
    lp = params['lp']
    desiredFP = params['fp']

    N1, N2, ea1, eb1, ea2, eb2, ls1, ls2, lb1, lb2 = self.calc(nk, lk, lp, ldCuckoo)

    if l3 and allowGateway or not alien:
      return 2 + (nbCuckoo - 1) / 2 + (1 + nbCuckoo) * N2 / nk
    else:
      return 5

  def getLvl1Ratio(self, x="lk"):
    def calc(nk, lk, lp, ld):
      N1, N2, ea1, eb1, ea2, eb2, ls1, ls2, lb1, lb2 = self.calc(nk, lk, lp, ld)
      return N1 / nk

    return [calc(nk if x is not "nk" else i, lk if x is not "lk" else i, lp if x is not "lp" else i, ldCuckoo)
            for i in range(globals()[x + "Min"], globals()[x + "Max"] + 1)]


class OthelloForwarder(Base):
  def calc(self, nk, lk, lp, ld):
    lsa = lp + (0 if l3 and allowGateway else ld)
    lsb = lp + (0 if l3 and allowGateway else ld)

    if alignSlotToByte:
      lsa = (lsa + 7) // 8 * 8
      lsb = (lsb + 7) // 8 * 8

    Ea = eaOthello if compact else 2 ** math.ceil(math.log2(nk * eaOthello)) / nk / eaOthello
    Eb = ebOthello if compact else 2 ** math.ceil(math.log2(nk * ebOthello)) / nk / ebOthello

    ma = Ea * nk
    mb = Eb * nk

    return ma, mb, Ea, Eb, lsa, lsb

  def calcMemoryFootprint(self, **params):
    nk = params['nk']
    lk = params['lk']
    lp = params['lp']
    desiredFP = params['fp']

    if l3 and allowGateway:
      ma, mb, Ea, Eb, lsa, lsb = self.calc(nk, lk, lp, 0)
      return ma * lsa + mb * lsb

    for ld in range(1, 100):
      ma, mb, Ea, Eb, lsa, lsb = self.calc(nk, lk, lp, ld)

      ea = math.exp(- nk / ma)
      eb = math.exp(- nk / mb)

      fp = 1 / (2 ** (ld - 1)) * (1 - ea) * (1 - eb)

      if fp <= desiredFP:
        break

    return ma * lsa + mb * lsb

  def getMemoryFootPrint(self, x="lk"):
    def calc(nk, lk, lp, ld):
      ma, mb, Ea, Eb, lsa, lsb = self.calc(nk, lk, lp, ld)
      return ma * lsa + mb * lsb

    return [calc(nk if x is not "nk" else i, lk if x is not "lk" else i, lp if x is not "lp" else i,
                 0 if allowGateway else ldOthello)
            for i in range(globals()[x + "Min"], globals()[x + "Max"] + 1)]

  def getRead(self, x="lk"):
    def calc(nk, lk, lp, ld):
      ma, mb, Ea, Eb, lsa, lsb = self.calc(nk, lk, lp, ld)

      if alignToCacheLine:
        return math.ceil(lk / lc) + 2
      else:
        lga = math.gcd(lc, lsa)
        lgb = math.gcd(lc, lsb)

        return math.ceil(lk / lc) + 2 + (lsa - lga) / lc + (lsb - lgb) / lc

    return [calc(nk if x is not "nk" else i, lk if x is not "lk" else i, lp if x is not "lp" else i,
                 0 if allowGateway else ldOthello)
            for i in range(globals()[x + "Min"], globals()[x + "Max"] + 1)]

  def getHashTimes(self, x="lk"):
    def calc(nk, lk, lp, ld):
      ma, mb, Ea, Eb, lsa, lsb = self.calc(nk, lk, lp, ld)
      ea = math.exp(- nk / ma)
      eb = math.exp(- nk / mb)

      if l3 and allowGateway or not alien:
        return 3
      else:
        return ea + (1 - ea) * eb * 2 + (1 - ea) * (1 - eb) * 3

    return [calc(nk if x is not "nk" else i, lk if x is not "lk" else i, lp if x is not "lp" else i,
                 0 if allowGateway else ldOthello)
            for i in range(globals()[x + "Min"], globals()[x + "Max"] + 1)]

  def calcHashTimes(self, **params):
    nk = params['nk']
    lk = params['lk']
    lp = params['lp']

    ma, mb, Ea, Eb, lsa, lsb = self.calc(nk, lk, lp, ldOthello)
    ea = math.exp(- nk / ma)
    eb = math.exp(- nk / mb)

    if l3 and allowGateway or not alien:
      return 3
    else:
      return ea + (1 - ea) * eb * 2 + (1 - ea) * (1 - eb) * 3

  def getFalsePositive(self, x="lk"):
    def calc(nk, lk, lp, ld):
      if allowGateway:
        return 1 if not l3 else 0

      ma, mb, Ea, Eb, lsa, lsb = self.calc(nk, lk, lp, ld)

      ea = math.exp(- nk / ma)
      eb = math.exp(- nk / mb)

      return 1 / (2 ** (ld - 1)) * (1 - ea) * (1 - eb)

    return [calc(nk if x is not "nk" else i, lk if x is not "lk" else i, lp if x is not "lp" else i,
                 0 if allowGateway else ldOthello)
            for i in range(globals()[x + "Min"], globals()[x + "Max"] + 1)]


if __name__ == '__main__':
  solutions = [BloomForwarder(), CuckooForwarder(), OthelloForwarder()]

  commonXTitles = ["lk", "lp"]

  yTitles = ["ExtraRead", "Lvl1RatioCuckoo", "MemoryFootPrint", "Read",
             "HashTimes", "FalsePositive"]

  vars = "nk", "lkMax", "lk", "lp", "ldCuckoo", "ldOthello"
  booleans = "l3", "alignToCacheLine", "alignSlotToByte", "alien", "allowGateway", "compact"
  floats = "targetFP",

  if not in_jupyter():
      fig = plt.figure()
      fig.set_figwidth(8)
      fig.set_figheight(1.6 * len(yTitles))

  def redraw():
    if in_jupyter():
      fig = plt.figure()
      fig.set_figwidth(8)
      fig.set_figheight(1.6 * len(yTitles))
    else:
      fig.clf()
    
    for row in range(len(yTitles), 0, -1):
      yTitle = yTitles[row - 1]

      lastWordInYTitle = camel_case_split(yTitle)[-1]
      tmpSolutions = []

      for solution in solutions:
        if lastWordInYTitle in solution.__class__.__name__:
          tmpSolutions.append(solution)

      if len(tmpSolutions) == 0:
        tmpSolutions = solutions
      else:
        yTitle = yTitle[0:-len(lastWordInYTitle)]

      for column in range(1, len(commonXTitles) + 1):
        xTitle = commonXTitles[column - 1]
        ax = fig.add_subplot(len(yTitles), len(commonXTitles), len(commonXTitles) * (row - 1) + column)
        if row == len(yTitles):
          ax.set_xlabel(xTitle, fontsize='x-small')
        else:
          plt.setp(ax.get_xticklabels(), visible=False)
        ax.set_ylabel(yTitle, fontsize='x-small')

        for solution in tmpSolutions:
          line, = ax.plot(range(globals()[xTitle + "Min"], globals()[xTitle + "Max"] + 1),
                          getattr(solution, "get" + yTitle)(xTitle))
    
    fig.tight_layout()
    
    if in_jupyter():
      plt.show()
    else:
      fig.canvas.draw()

  if in_jupyter():
    from ipywidgets import *
    import ipywidgets as widgets

    def sync(**kwargs):
      for var in kwargs.keys():
        try:
          globals()[var] = kwargs[var]
        except Exception as e:
          print(e)


    left = []
    right = []
    d = {}
    for var in vars:
      slider = widgets.IntSlider(description=var, min=globals()[var + "Min"], max=globals()[var + "Max"], step=1,
                                 value=globals()[var])
      d[var] = slider
      left.append(slider)

    for var in floats:
      slider = widgets.FloatLogSlider(description=var, min=globals()[var + "Min"], max=globals()[var + "Max"], step=1,
                                      value=globals()[var])
      d[var] = slider
      left.append(slider)

    for var in booleans:
      checkbox = widgets.Checkbox(description=var, value=globals()[var])
      d[var] = checkbox
      right.append(checkbox)

    ui = HBox((VBox(left), VBox(right)))


    def update(nk, lkMax, lk, lp, ldCuckoo, ldOthello, l3, alien, alignToCacheLine, alignSlotToByte,
               allowGateway, targetFP, compact):
      sync(**locals())

      d["lk"].max = lkMax

      if lk > lkMax:
        globals()["lk"] = lkMax
        d["lk"].value = lkMax

      if nk > 2 ** lk:
        lkMin = math.ceil(math.log2(nk))
        globals()["lk"] = lkMin
        d["lk"].min = lkMin

      redraw()

    display(ui, widgets.interactive_output(update, d))
  else:
    from matplotlib.widgets import *

    controlFig = plt.figure()


    def checkChanged(label):
      globals()[label] = not globals()[label]
      redraw()


    rax = plt.axes([0.05, 0.05, 0.9, 0.4])

    check = CheckButtons(rax, booleans, tuple(globals()[name] for name in booleans))
    check.on_clicked(checkChanged)

    sliders = {}


    def sliderChanged(val):
      for var in vars:
        globals()[var] = int(sliders[var].val)

      for var in floats:
        globals()[var] = sliders[var].val

      if lk > lkMax:
        sliders["lk"].val = lkMax

      elif nk > 2 ** lkMin:
        min = math.ceil(math.log2(nk))
        sliders["lk"].valmin = min
        globals()["lkMin"] = min

        if sliders["lk"].val < min:
          sliders["lk"].val = min
      else:
        redraw()
        return

      sliderChanged(val)


    for var in vars + floats:
      sax = plt.axes([0.15, 0.55 + len(sliders) * 0.05, 0.7, 0.035], facecolor='lightgoldenrodyellow')
      slider = Slider(sax, var, globals()[var + "Min"], globals()[var + "Max"], valinit=globals()[var])
      slider.on_changed(sliderChanged)
      sliders[var] = slider

    sliders["lk"].slidermax = sliders['lkMax']
    redraw()

    plt.show()


Output()